In [1]:
from dataeval.detectors.ood.ae_torch import OOD_AE


2024-11-11 19:19:10.843075: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-11 19:19:10.845955: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-11 19:19:10.851845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731352750.862111   67505 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731352750.864986   67505 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

In [1]:
import numpy as np

from dataeval.detectors.ood.ae_torch import OOD_AE
from dataeval.utils.torch.datasets import MNIST
from dataeval.utils.torch.models import AriaAutoencoder

2024-11-11 18:58:16.101650: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-11 18:58:16.174901: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-11 18:58:16.231474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731351496.280350   64300 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731351496.295694   64300 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

In [ ]:
def get_default_encoder_net(input_shape: tuple[int, int, int], encoding_dim: int):
    return Sequential(
        [
            InputLayer(input_shape=input_shape),
            Conv2D(64, 4, strides=2, padding="same", activation=relu),
            Conv2D(128, 4, strides=2, padding="same", activation=relu),
            Conv2D(512, 4, strides=2, padding="same", activation=relu),
            Flatten(),
            Dense(encoding_dim),
        ]
    )


def get_default_decoder_net(input_shape: tuple[int, int, int], encoding_dim: int):
    return Sequential(
        [
            InputLayer(input_shape=(encoding_dim,)),
            Dense(4 * 4 * 128),
            Reshape(target_shape=(4, 4, 128)),
            Conv2DTranspose(256, 4, strides=2, padding="same", activation=relu),
            Conv2DTranspose(64, 4, strides=2, padding="same", activation=relu),
            Flatten(),
            Dense(math.prod(input_shape)),
            Reshape(target_shape=input_shape),
        ]
    )

In [2]:
# Load in the training mnist dataset and use the first 4000
train_ds = MNIST(root="./data/", train=True, download=True, size=4000, unit_interval=True, channels="channels_first")

# Split out the images and labels
images, labels = train_ds.data, train_ds.targets
input_shape = images[0].shape

Files already downloaded and verified


In [ ]:
#%%
detectors = [
    OOD_AE(AriaAutoencoder(channels=1)),
]
# detectors = [AriaAutoencoder(channels=1)]

In [3]:
import torch
import os

new_ood = OOD_AE(AriaAutoencoder(channels=1))

# model = TheModelClass(*args, **kwargs)
# optimizer = TheOptimizerClass(*args, **kwargs)
checkpoint_path = os.getcwd()+'/OOD_20241111_0942.pt'

checkpoint = torch.load(checkpoint_path, weights_only=True)
new_ood.model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# new_ood.model.eval()
detectors = [new_ood]

# Gradients collapsing to zero....why?

model params are being stepped, but steps drop to zero after 165 iterations


In [4]:
# 50 epochs about 10 minutes
for detector in detectors:
    print(f"Training {detector.__class__.__name__}...")
    detector.fit(images, threshold_perc=99, epochs=1, verbose=True)

Training OOD_AE...
Epoch 0...
loss: 0.0037996775936335325, |grad|: 0.09965380710799894
loss: 0.002683198545128107, |grad|: 0.09452687117641327
loss: 0.005245742853730917, |grad|: 0.12487432802683006
loss: 0.005218362435698509, |grad|: 0.07444694275288781
loss: 0.0027823697309941053, |grad|: 0.09393064317296468
loss: 0.0035670222714543343, |grad|: 0.08593201273081992
loss: 0.0016407860675826669, |grad|: 0.044660380380051795
loss: 0.009273688308894634, |grad|: 0.15486524868035678


/workspaces/dataeval/src/dataeval/detectors/drift/torch.py:127: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_tmp = model(torch.tensor(x_batch).to(torch.float32))


In [ ]:
# import torch
# model = detectors[0].model
# recon = model(torch.from_numpy(images).to(torch.float32))

In [ ]:
# import matplotlib.pyplot as plt
# fig, axs = plt.subplots(1,2)

# i = 10
# axs[0].imshow(images[i,0,:,:])
# axs[1].imshow(recon[i,0,:,:].detach())

In [ ]:
# plt.plot(recon[0,0,:,:].detach().reshape(-1) , recon[199,0,:,:].detach().reshape(-1))

In [9]:
corruption = MNIST(
    root="./data",
    train=True,
    download=False,
    size=2000,
    unit_interval=True,
    channels="channels_first",
    corruption="shot_noise",
)
corrupted_images = corruption.data

Files already downloaded and verified


In [ ]:
# from importlib import reload
# import dataeval.detectors.drift.torch

# reload(dataeval.detectors.drift.torch)

In [10]:
print([(type(detector).__name__, np.mean(detector.predict(images).is_ood)) for detector in detectors])
print([(type(detector).__name__, np.mean(detector.predict(corrupted_images).is_ood)) for detector in detectors])

[('OOD_AE', 0.01)]
[('OOD_AE', 0.887)]


In [ ]:
dir(detector)

In [ ]:
import os
os.getcwd()

In [ ]:
# epoch = 250

# # torch.save({
# #             'epoch': epoch,
# #             'model_state_dict': detector.model.state_dict(),
# #             'optimizer_state_dict': detector. optimizer.state_dict(),
# #             'loss': loss,
# #             ...
# #             }, PATH)

# checkpoint_path = os.getcwd()+'/OOD_20241111_0942.pt'

# torch.save({
#             'epoch': epoch,
#             'model_state_dict': detector.model.state_dict(),
#             }, checkpoint_path)

In [ ]:
import torch
import os

new_ood = OOD_AE(AriaAutoencoder(channels=1))

# model = TheModelClass(*args, **kwargs)
# optimizer = TheOptimizerClass(*args, **kwargs)
checkpoint_path = os.getcwd()+'/OOD_20241111_0942.pt'

checkpoint = torch.load(checkpoint_path, weights_only=True)
new_ood.model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
# loss = checkpoint['loss']

new_ood.model.eval()

In [ ]:
for n, o in zip(new_ood.model.parameters(), detector.model.parameters()):
    print((n==o).all())

In [ ]:
detectors = [new_ood]